<a href="https://colab.research.google.com/github/salarbalou/photothermal_heating/blob/main/Photothermal_Study_of_CQDs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import plotly.io as pio
pio.templates.default= 'plotly_dark'
from plotly.subplots import make_subplots

In [6]:
#for blue 435nm maxpower=4W laser:
m, b = 7.025524475524473, -66.07575757575763
# dc is arduino board pwm pin duty cycle
dc = np.linspace(10, 150, 15) # at 150 dc we will reach 1W of power for laser

In [ ]:
t = #time (s)
T= #temperature of the solution (C)
Tw = #temperature of the environment (C) (needs to be measured by another K type thermocouple)
Iout = #measured using powermeter: f(CCQD, L_laser) (W)



S_PCR_tube = #surface area of the solution at which laser beam is passing through:[PCR_tube, paper] (m**2)
S_paper = #surface are of the paper at which laser beam is covering:[PCR_tube, paper]  (m**2)
m = rho * V #mass of solution (Kg)

In [13]:
Iin = m*dc + b #input power of the laser: f(dc) (W)
h = 1000 #heat transfor coefficient of solution with its surrounding (W/(m**2)*C)

rho  = 1000 #density of solution (kg/m**3)
V = 1e-8 #volume of the solution (m**3)
Cp = 4182 #heat capacity of water (J/Kg*C)

In [14]:
def der(T, t, dt = 1):
  return T(t+dt)-T(t)/dt

In [ ]:
R  = der(T[T, t[:60]) #heating rate (C/s) #heating solution for 60 seconds then turn off laser
C = der(T, t[60:]) #cooling rate (C/s) #turning off laser for 60 seconds to measure the cooling rate

In [46]:
t = np.linspace(0, 120, 120)
nx = 100
nt = 60 #temperature rise is evaluated in 60 seconds
T = np.full(nx, 22)
T = np.random.normal(loc = 50, scale =1, size = t.size)

In [51]:
fig = make_subplots()
fig.add_scatter(x = np.arange(nx), y = np.full(nx, min(T)), name = '0 s')
for i in range (nt+1):
  T = T = np.random.normal(loc = 50, scale =1, size = t.size)
  fig.add_scatter(x = np.arange(nx), y = T, name = f'{i} s')
fig.update_layout(height = 800, width = 1000)
fig.show()